<link rel="stylesheet" href="/site-assets/css/gemma.css">
<link rel="stylesheet" href="https://fonts.googleapis.com/css2?family=Google+Symbols:opsz,wght,FILL,GRAD@20..48,100..700,0..1,-50..200" />

##### Copyright 2024 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Using LIT with Gemma

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemma/docs/lit_gemma"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on Generative AI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/gemma/docs/lit_gemma.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/gemma/docs/lit_gemma.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://codelabs.developers.google.com/codelabs/responsible-ai/lit-gemma"><img src="https://www.tensorflow.org/images/codelabs_logo.png" height="24" width="48" />Learn in Codelabs</a>
  </td>
</table>

Generative AI products are relatively new and their behaviors can vary more than
earlier forms of software. This makes it important to probe the machine learning
models being used, examine examples of the model's behavior and investigate
surprises.

The Learning Interpretability Tool (LIT; [website][lit-web], [GitHub][lit-gh])
is a platform for debugging and analyzing ML models to understand why and how
they behave the way they do.

Here, you'll learn how to setup LIT to get more out of Google's 
[Gemma model][gemma] by using the Sequence Salience module to analyze different
prompt engineering approaches.

[lit-web]: https://pair-code.github.io/lit
[lit-gh]: https://github.com/PAIR-code/lit
[gemma]: https://ai.google.dev/gemma
[keras-nlp]: https://keras.io/keras_nlp/

# Setting up LIT to Debug Gemma Prompts

*Note: you may see some warnings of the form*

```
ERROR: pip's dependency resolver does not currently take into account all the 
packages that are installed. This behaviour is the source of the following 
dependency conflicts.
bigframes 0.21.0 requires scikit-learn>=1.2.2, but you have scikit-learn 1.0.2 
which is incompatible.
google-colab 1.0.0 requires ipython==7.34.0, but you have ipython 8.14.0 
which is incompatible.
```

*These are safe to ignore.*

## Install LIT and Keras NLP

This notebook uses the KerasNLP implementation of Gemma (more on how to 
configure this below). You will need a recent version of `keras` (3.0+) 
`keras-nlp` (0.12+) and `lit-nlp` (1.2+), and a Kaggle account to download the 
base model.

In [ ]:
# Keras is included in Colab runtimes, but needs to be updated to to v3.0+.
# LIT and Keras NLP are not icnldued by default and must be installed.
# Running this cell may require you to restart your session to ensure the newer
# packages are imported correctly.
! pip install -q -U "keras >= 3.0, <4.0" "keras-nlp >= 0.14" "lit-nlp >= 1.2"

### Kaggle Access

KerasNLP stores their pre-trained model weights on Kaggle. The
[`kagglehub` package](https://github.com/Kaggle/kagglehub#authenticate) is used
to autheticate with this service. Be sure to also accept the license agreement 
for [Gemma](https://www.kaggle.com/models/keras/gemma) from your Kaggle account.

See the Appendix at the end for more information on how to set up a Kaggle
account.

In [ ]:
import kagglehub

kagglehub.login()

## Configuring LIT

LIT provides a function, `make_notebook_widget()` for configuring our prompt 
debugging tools in a notebook context. 

LIT provides a dataset of sample prompts that accompany the tutorial linked 
later in this document.

See the comments below for configuring the widget to use different models and/or
datasets.

In [ ]:
from lit_nlp.examples.prompt_debugging import notebook as lit_pdbnb

# The following function initializes a LIT Notebook Widget. It's configured by
# two required positional arguments:
#
# * `datasets_config`: A list of strings containing the dataset names and
#       paths to load from, as "dataset:path", where path can be a URL or a
#       local file path. The example below uses a special value,
#       `sample_prompts`, to load the example prompts provided in the LIT
#       distribution; no other special values are supported.
# * `models_config`: A list of strings containing the model names and paths to
#       load from, as "model:path", where path can be a URL, a local file path,
#       or the name of a preset for the configured deep learning framework.
#
# LIT supports salience computation for KerasNLP and Hugging Face Transformers
# models running on TensorFlow or PyTorch. Note that all models passed to the
# `models_config` parameter will be loaded using the same framework and runtime.
# You can cofnigre these with the following keywork arguments.
#
# * `dl_framework`: Must be one of "kerasnlp" or "transformers".
# * `dl_runtime`: Must be one of "tensorflow" or "torch".
#
# Changing the `dl_framework` value will affect the authentication method used
# to access Gemma model weights.

lit_widget = lit_pdbnb.make_notebook_widget(
    ['sample_prompts'],
    ["gemma_2b_it:gemma_1.1_instruct_2b_en"],
    dl_framework="kerasnlp",
    dl_runtime="tensorflow",
    batch_size=1,
    max_examples=5,
    precision="bfloat16",
)

Now you can render the UI in a Colab cell.

In [ ]:
lit_widget.render()

<IPython.core.display.Javascript object>

# Prompt Debugging with Sequence Salience

Text-to-text large language models (LLMs), such as Gemma, take an input sequence 
in the form of [tokenized][tokenization] text and generate new tokens that are 
logical follow-ons or completions.

[Salience methods][salience-explorable] allow you to inspect which parts of an 
input are important to the model for different parts of its generated output. 
LIT's [Sequence Salience module][lit-seq-sal] extends these methods to explain 
the importance of sequences at multiple levels of granularity: from tokens to 
words to sentences and beyond.

You can use LIT in the cell above to play around with the Sequence Salience 
module on your own. For a more guided learning experience, you can follow long 
with the [_Prompt Debugging with Sequence Salience_ tutorial][seq-sal-tutorial]
right in this Colab.

For even more academic and techncial information on how Sequence Salience works,
check out [our paper][seq-sal-paper].


[lit-seq-sal]: https://pair-code.github.io/lit/documentation/components.html#sequence-salience
[salience-explorable]: https://pair.withgoogle.com/explorables/saliency/
[seq-sal-paper]: https://arxiv.org/abs/2404.07498
[seq-sal-tutorial]: https://pair-code.github.io/lit/tutorials/sequence-salience/
[tokenization]: https://arxiv.org/pdf/1808.06226.pdf


# Appendix: Accessing Gemma on Kaggle Hub

This notebook uses the KerasNLP implementation of Gemma in this document. 
KerasNLP stores their pre-trained model weights on Kaggle, and Gemma requires 
authentication and license acknowledgement to access those weights.

The following instruction walk you through how to set up a Kaggle account and 
authenticate with Kaggle using the `kagglehub` package.

1.  Create a Kaggle account if you don't have one
    * Go to: https://www.kaggle.com/account/login?phase=startRegisterTab
    * Use whichever registation method you prefer to set up your account.
1. Request access to Gemma
    * Make sure you're logged into Kaggle using the account above
    * Go to the consent page: https://www.kaggle.com/models/google/gemma/license/consent
    * Select the "Verify via Kaggle Account" option (the default selection) and click next
    * Complete the consent form (first name and last name fields at the top)
    * Acknowledge the policy using the checkboxes at the bottom
    * Click the "Accept" button at the bottom to be granted access
    * This should redirect you to the model page (https://www.kaggle.com/models/google/gemma)
1. Create an API token
    * Make sure you're logged into Kaggle using the account you created above
    * Got to the Settings page: https://www.kaggle.com/settings
    * Scroll down to the API section
    * Use the "Create New Token" button to trigger token generation
    * Use the on-screen menu to save the JSON file, named kaggle.json, the service generates to your machine
    * The JSON file is an object with two properties, username and key, you'll need both to authenticate with their service later
1. Use your API token credentials to authenticate with kagglehub in Colab
    * Go to the LIT Sequence Saleince Colab: https://colab.sandbox.google.com/github/google/generative-ai-docs/blob/main/site/en/gemma/docs/lit_gemma.ipynb#scrollTo=yKw8gDsh_nVR
    * Conntect to a GPU runtime
    * For Gemma 2B you can use the free-tier T4 runtime
    * For Gemma 7B you will need pre-paid Colab compute credits or a Colab Pro account to use a V100, L4, or A100 GPU
    * Run the `kagglehub` code cell to display an HTML form that asks for your username and a token
    * Copy the `username` field from the `kaggle.json` file you downloaded in the previous step and paste it into the `username` field in the form
    * Copy the `key` field from the `kaggle.json` file you downloaded in the previous step and paste it into the `token` field in the form
    * Click the login button to save these credentials in your runtime

You will need to repeat the last step any time the Colab runtime is disconnected, as disconnection clears the cache the credentials are stored in.